In [1]:
import rospy
from rospy import loginfo, logwarn, logerr
import tf2_ros
import tf
import cv2
import numpy as np
from sensor_msgs.msg import PointCloud2, PointField, Image
from sensor_msgs import point_cloud2 as pc2
from cv_bridge import CvBridge
bridge = CvBridge()
from matplotlib import pyplot as plt
from geometry_msgs.msg import Point, Twist
from std_msgs.msg import String, Header
from fields_ignition_msgs.msg import Detection, Detections
import itertools
import struct

In [2]:
rospy.init_node('follow')
loginfo('running...')
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

def get_param(name, default):
    try:
        value = rospy.get_param(name, default)
    except ConnectionRefusedError:
        value = default
    print('param "{}" = "{}"'.format(name, value))
    return value

CMD_VEL_TOPIC = get_param('~cmd_vel_topic', '/costar_husky_sensor_config_1/cmd_vel')

[INFO] [1666068246.583513, 0.000000]: running...
param "~cmd_vel_topic" = "/costar_husky_sensor_config_1/cmd_vel"


In [3]:
pub_cmd_vel = rospy.Publisher(CMD_VEL_TOPIC, Twist, queue_size=1)
pub_cmd_vel

In [4]:
robot_initial_pose = (0,0,0,0,0,0)

In [5]:
def move():
    # Starts a new node
    rospy.init_node('speed_controller', anonymous=True)
    velocity_publisher = rospy.Publisher('/costar_husky_sensor_config_1/cmd_vel', Twist, queue_size=10)
    vel_msg = Twist()

    #Receiveing the user's input
    print("Let's move your robot")
    speed = int(input("Input your speed:"))
    distance = int(input("Type your distance:"))
    isForward = bool(input("Foward?: "))#True or False

    #Checking if the movement is forward or backwards
    if(isForward):
        vel_msg.linear.x = abs(speed)
    else:
        vel_msg.linear.x = -abs(speed)
    #Since we are moving just in x-axis
    vel_msg.linear.y = 0
    vel_msg.linear.z = 0
    vel_msg.angular.x = 0
    vel_msg.angular.y = 0
    vel_msg.angular.z = 0

    while not rospy.is_shutdown():

        #Setting the current time for distance calculus
        t0 = rospy.Time.now().to_sec()
        current_distance = 0

        #Loop to move the turtle in an specified distance
        while(current_distance < distance):
            #Publish the velocity
            velocity_publisher.publish(vel_msg)
            #Takes actual time to velocity calculus
            t1=rospy.Time.now().to_sec()
            #Calculates distancePoseStamped
            current_distance= speed*(t1-t0)
        #After the loop, stops the robot
        vel_msg.linear.x = 0
        #Force the robot to stop
        velocity_publisher.publish(vel_msg)

In [ ]:
move()

In [6]:
def get_robot_pose():
    try:
        print(1)
        tf_robot =  tfBuffer.lookup_transform('field', 'costar_husky_sensor_config_1', rospy.Time(), rospy.Duration(1.0))
        print(2)
        yaw = tf.transformations.euler_from_quaternion([
            tf_robot.transform.rotation.x,
            tf_robot.transform.rotation.y,
            tf_robot.transform.rotation.z,
            tf_robot.transform.rotation.w,
        ])[2]
        return np.array([tf_robot.transform.translation.x, tf_robot.transform.translation.y]), yaw
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
        rospy.logwarn('Navigator: Can\'t find transform to sensor')
        rospy.sleep(1)
        return None


In [7]:
class TracePoints():
    def __init__(self, topic="debug_points", frame_id="field"):
        self.publisher = rospy.Publisher(topic, PointCloud2, queue_size=1)
        self.frame_id = frame_id

        self.fields = [PointField('x', 0, PointField.FLOAT32, 1),
                       PointField('y', 4, PointField.FLOAT32, 1),
                       PointField('z', 8, PointField.FLOAT32, 1),
                       # PointField('rgb', 12, PointField.UINT32, 1),
                       PointField('rgba', 12, PointField.UINT32, 1),
                       ]
        self.reset()

    def reset(self):
        self.points = []

    def add_point(self, point, color):
        x = point[0]
        y = point[1]
        z = point[2]
        r = int(color[0] * 255.0)
        g = int(color[1] * 255.0)
        b = int(color[2] * 255.0)
        a = 255
        rgb = struct.unpack('I', struct.pack('BBBB', b, g, r, a))[0]
        pt = [x, y, z, rgb]
        self.points.append(pt)

    def publish(self):
        header = Header()
        header.frame_id = self.frame_id
        msg = pc2.create_cloud(header, self.fields, self.points)
        self.publisher.publish(msg)

In [8]:
path = [
    np.array([-1, -1]),
    np.array([0, -.75]),
    np.array([.7, -0.2]),
    np.array([1, 1]),
    np.array([1, 0.9*10]),
    np.array([1.3, 0.9*11.5]),
    np.array([2, 0.9*12]),
    np.array([2.7, 0.9*11.5]),
    np.array([3, 0.9*10]),
    np.array([3, -1]),
    np.array([5, -1]),
    np.array([5, 0.9*12]),
]
#path = np.concatenate([np.linspace(path[i-1], path[i], np.sqrt(np.sum((path[i-1] - path[i])**2))*3+1)[1:] for i in range(1, len(path))])
tp = TracePoints('path')
for x, y in path:
    tp.add_point([x, y, 0], [0,0,1])
tp.publish()


In [9]:
def step_robot():
    robot_xy, robot_yaw = get_robot_pose()
    path_distances = path - robot_xy
    path_distances = np.hypot(path_distances[:,0], path_distances[:,1])
    min_dist = 0.5
    target_points_idx = np.max(np.argsort(path_distances)[:5])
    if path_distances[target_points_idx] < min_dist:
        target_points_idx += 1
    target_xy = path[target_points_idx]

    tp.reset()
    for x, y in path:
        color = [1,0,0] if x == target_xy[0] and y == target_xy[1] else [0,0,1]
        tp.add_point([x, y, 0], color)
    tp.publish()

    print(target_xy  , robot_xy)
    x_diff, y_diff = target_xy - robot_xy
    angle = np.arctan2(y_diff, x_diff)
    yaw_diff = (((angle - robot_yaw) + np.pi) % (2*np.pi)) - np.pi
    print(y_diff, x_diff, yaw_diff)

    max_yaw_diff = 0.8
    cmd_msg = Twist()
    if np.abs(yaw_diff) > max_yaw_diff:
        cmd_msg.angular.z = yaw_diff * 1.2
    else:
        cmd_msg.angular.z = yaw_diff
        cmd_msg.linear.x = 1.0 - yaw_diff
    print(cmd_msg)
    pub_cmd_vel.publish(cmd_msg)

In [10]:
#rospy.init_node('speed_controller', anonymous=True)
try:
    rate = rospy.Rate(10)
    while not rospy.is_shutdown():
        step_robot()
        rate.sleep()
except rospy.ROSInterruptException:
    pass

1


[WARN] [1666068260.194559, 11.627000]: Navigator: Can't find transform to sensor


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
np.array([1,1])  np.array([1,1])